In [1]:
import numpy as np

In [2]:
class Dynamics:
    pass

In [3]:
hw5dynamics=Dynamics()
hw5dynamics.kappa = 0.472
hw5dynamics.alpha = 4.4
hw5dynamics.sigma = 0.368
hw5dynamics.S0 = 106.9
hw5dynamics.r = 0.05

In [4]:
class Contract:
    pass

In [5]:
hw5contract=Contract()
hw5contract.K1 = 103.2
hw5contract.T1 = 0.5
hw5contract.T2 = 0.75

In [6]:
class MC:
    pass

In [7]:
hw5MC=MC()
hw5MC.N = 100   # Number of timesteps on each path
hw5MC.M = 1000  # Number of paths.  Change this if necessary.
hw5MC.epsilon = 0.01  # For the dC/dS calculation
hw5MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [10]:
def pricer_oilcall_OU_MC(contract,dynamics,MC):
    
    K1,T1,T2 = contract.K1,contract.T1,contract.T2
    kappa,alpha,sigma,S0,r = dynamics.kappa,dynamics.alpha,dynamics.sigma,dynamics.S0,dynamics.r
    N,M,epsilon = MC.N,MC.M,MC.epsilon

    np.random.seed(MC.seed)  #seed the random number generator

    deltat = T1/N
    S0_delta = S0+epsilon
    t = np.linspace(0,T1,num = N+1)
    
    def path(S0,dBt):
        X0 = np.log(S0)
        Xt = [X0]
        for i in range(N):
            currX = Xt[i] + kappa*(alpha-Xt[i])*deltat + sigma*dBt[i]
            Xt.append(currX)
        discount = np.exp(-kappa*(T2-t))
        Ft = np.exp(discount*Xt+(1-discount)*alpha+sigma**2/(4*kappa)*(1-discount**2))
        callpayoff = np.exp(-r*T1)*np.maximum(Ft[-1] - K1 , 0)
        return callpayoff
        
    payoffs1 = []
    payoffs2 = []
    
    for _ in range(M):
        dBt = np.sqrt(deltat)*np.random.normal(0,1,size = N)
        payoff1 = path(S0,dBt)
        payoff2 = path(S0_delta,dBt)
        payoffs1.append(payoff1)
        payoffs2.append(payoff2)

    call_price = np.average(payoffs1)
    standard_error = np.std(payoffs1)/np.sqrt(M)
    call_delta = (np.average(payoffs2)-call_price)/epsilon
        
    return(call_price, standard_error, call_delta)
        

In [11]:
(call_price, standard_error, call_delta) = pricer_oilcall_OU_MC(hw5contract,hw5dynamics,hw5MC)

In [12]:
print(call_price, standard_error, call_delta)

8.280892135445974 0.43571675038027846 0.35561444892682204


In [13]:
#### 1e

S0 = hw5dynamics.S0
r = hw5dynamics.r
T2 = hw5contract.T2
kappa = hw5dynamics.kappa
alpha = hw5dynamics.alpha
sigma = hw5dynamics.sigma

S0**(-1+np.exp(-kappa*T2))*np.exp(-r*T2-kappa*T2+(1-np.exp(-kappa*T2))*alpha+sigma**2/(4*kappa)*(1-np.exp(-2*kappa*T2)))

0.6465109164314246

In [14]:
#### 1f


call_delta/(S0**(-1+np.exp(-kappa*T2))*np.exp(-r*T2-kappa*T2+(1-np.exp(-kappa*T2))*alpha+sigma**2/(4*kappa)*(1-np.exp(-2*kappa*T2))))

0.550051731360892

In [15]:
#### 1g

K1 = hw5contract.K1
T1 = hw5contract.T1

disc = np.exp(-kappa*T2)
F0 = np.exp(disc*(np.log(S0))+(1-disc)*alpha+sigma**2/(4*kappa)*(1-disc**2))
f0 = (F0-K1)*np.exp(-r*T2)
v0 = 4000*f0+1000*call_price*np.exp(-r*(T2-T1))
v0

4442.188090633092